In [107]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [125]:
print_to_excel = True

In [122]:
days = export_from_RISKCUSTOM(f"""select distinct "reportDate"
from "RISKACCESS"."unifiedOcp" 
where "sourceTable" in ('unifiedDealsBrazil_externalPurchasesRecap', 'unifiedDealsBrazil_externalSalesOrders','unifiedDealsBrazil_purchasesRecapBRHRG','unifiedDealsBrazil_sapSalesOrders')
order by "reportDate"                    
""")
days.head(5)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,reportDate
0,2024-09-01
1,2024-09-02
2,2024-09-03
3,2024-09-04
4,2024-09-05


In [123]:
query = f"""select * 
from "RISKACCESS"."unifiedOcp" 
where "sourceTable" in ('unifiedDealsBrazil_externalPurchasesRecap', 'unifiedDealsBrazil_externalSalesOrders','unifiedDealsBrazil_purchasesRecapBRHRG','unifiedDealsBrazil_sapSalesOrders')
and "reportDate" = TO_DATE('{days.iloc[0,:].astype(str).item()}', 'YYYY-MM-DD')
--fetch first 10 rows only"""
data_OCP_export = export_from_RISKCUSTOM(query)
data_OCP_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(18385, 40)

In [124]:
data_group = data_OCP_export.groupby(as_index=False, dropna=False, by=['salesUnitsId','sourceTable','internalFlag'])['volumeUsd'].agg('sum')
data_group = data_group[data_group.salesUnitsId.isin(['BRFTO','BRHRG'])]
data_group
# unifiedDealsBrazil_externalPurchasesRecap - сделки от FTO на покупку, если котрагент внутренний, то в таблицу идет еще и встречная сделка
# сделки на покупку и продажу (на покупку с минусом - должны заплатить), sales - на продажу (ожидаем получить деньги)

,salesUnitsId,sourceTable,internalFlag,volumeUsd
4,BRFTO,unifiedDealsBrazil_externalPurchasesRecap,External,-3.712360e+08
5,BRFTO,unifiedDealsBrazil_externalPurchasesRecap,Internal,-1.785141e+08
6,BRFTO,unifiedDealsBrazil_externalSalesOrders,External,9.333603e+14
7,BRHRG,unifiedDealsBrazil_purchasesRecapBRHRG,External,-1.576452e+08
8,BRHRG,unifiedDealsBrazil_purchasesRecapBRHRG,Internal,-7.740332e+07
9,BRHRG,unifiedDealsBrazil_sapSalesOrders,External,2.399938e+08
10,BRHRG,unifiedDealsBrazil_sapSalesOrders,Internal,3.245347e+06


In [130]:
# FTO
# Pur
pur_str = '''AN 0
AS 208
MAP 87
MOP G 209
NK 13
NP 40
NPK MOP 102
SSP 116
TSP 60
UREA GRAN 214
OTHER 14'''
# Sales
sales_str = """AN -13
AS -228
CAN -5
MAP -139
MOP G -348
NK -25
NP -7
NPK MOP -46
NPK OTHER -11
NPS -25
SSP -113
TSP -48
UREA GRAN -217
OTHER -22
"""
prices_str = """MOP G	12	$283 
UREA GRAN	-62	$375 
AN	8	$205 
AS	22	$183 
CAN	2	$246 
MAP	-42	$612 
SSP	50	$187 
TSP	27	$440 
NP	35	$564 
PHOSPHATE ROCK	2	$137 
NPK 21:1:21 [MOP] COMP	-3	$337 
NPK 19:4:19 [MOP] COMP	1	$297 
NPK 15:15:15 [MOP] COMP	2	$394 
NPK 16:16:16 [MOP] COMP	8	$358 
NPS	1	$468 
CN WS (19CA) GRAN	-0	$403 
OTHER	40	$462"""
# HRG
# Pur
pur_str_hrg = '''AN 67
AS 133
CAN 7
MAP 71
MOP G 219
NP 20
NPK MOP 49
SSP 26
TSP 1
UREA GRAN 128
OTHER 14'''
# Sales
sales_str_hrg = """AN -91
AS -74
CAN -10
MAP -54
MOP G -154
MOP S -5
NK -1
NP -2
NPK MOP -40
NPS 12:40 -15
SSP -28
TSP -9
UREA GRAN -78
OTHER -19
"""
prices_str_hrg = """MOP G    113    $289 
UREA GRAN    39    $333 
AN    45    $227 
AS    94    $187 
CAN    6    $233 
MAP    58    $602 
SSP    16    $218 
TSP    -3    $477 
NP X:X BLEND    22    $510 
PHOSPHATE ROCK    11    $90 
NPK 21:1:21 [MOP] COMP    0    $286 
NPK 19:4:19 [MOP] COMP    -5    $395 
NPK 15:15:15 [MOP] COMP    7    $335 
NPK 16:16:16 [MOP] COMP    0    $512 
NPS 12:40    5    $457 
CN WS (19CA) GRAN    8    $464 
OTHER    28    $400"""



In [139]:
data_pur = pd.DataFrame([x.rsplit(' ', 1) for x in pur_str.split('\n')]).rename(columns={0:'BRFTO',1:'Purchases'})
data_sales = pd.DataFrame([x.rsplit(' ', 1) for x in sales_str.split('\n')]).rename(columns={0:'BRFTO',1:'Sales'})
data_merge_BRFTO = data_sales.merge(data_pur, how='outer', on='BRFTO').loc[1:,:]

data_prices = pd.DataFrame([x.split('\t') for x in prices_str.split('\n')]).rename(columns={0:'BRFTO',2:'Prices'}).drop(columns=1)
data_prices.Prices = data_prices.Prices.str[1:4]
data_merge = data_prices.merge(data_merge_BRFTO, how='outer', on='BRFTO')

# data_merge = data_merge.fillna(0)
data_merge[['Prices','Sales','Purchases']] = data_merge[['Prices','Sales','Purchases']].astype(float)
data_merge['amount_Purchases'] = data_merge.Prices * data_merge.Purchases *1000
data_merge['amount_Sales'] = data_merge.Prices * data_merge.Sales*1000

data_merge.loc[len(data_merge)] = {'BRFTO':'Total','Prices':data_merge.Prices.sum(),'Sales':data_merge.Sales.sum(),'Purchases':data_merge.Purchases.sum(),'amount_Purchases':data_merge.amount_Purchases.sum(),'amount_Sales':data_merge.amount_Sales.sum()}

In [142]:
data_pur_hrg = pd.DataFrame([x.rsplit(' ', 1) for x in pur_str_hrg.split('\n')]).rename(columns={0:'BRHRG',1:'Purchases'})
data_sales_hrg = pd.DataFrame([x.rsplit(' ', 1) for x in sales_str_hrg.split('\n')]).rename(columns={0:'BRHRG',1:'Sales'})
data_merge_BRHRG = data_sales_hrg.merge(data_pur_hrg, how='outer', on='BRHRG').loc[1:,:]

data_prices_hrg = pd.DataFrame([x.split('   ') for x in prices_str_hrg.split('\n')]).rename(columns={0:'BRHRG',2:'Prices'}).drop(columns=1)
data_prices_hrg.Prices = data_prices_hrg.Prices.str[2:5]
data_merge_hrg = data_prices_hrg.merge(data_merge_BRHRG, how='outer', on='BRHRG')

# data_merge = data_merge.fillna(0)
data_merge_hrg[['Prices','Sales','Purchases']] = data_merge_hrg[['Prices','Sales','Purchases']].astype(float)
data_merge_hrg['amount_Purchases'] = data_merge_hrg.Prices * data_merge_hrg.Purchases*1000
data_merge_hrg['amount_Sales'] = data_merge_hrg.Prices * data_merge_hrg.Sales*1000

data_merge_hrg.loc[len(data_merge_hrg)] = {'BRHRG':'Total','Prices':data_merge_hrg.Prices.sum(),'Sales':data_merge_hrg.Sales.sum(),'Purchases':data_merge_hrg.Purchases.sum(),'amount_Purchases':data_merge_hrg.amount_Purchases.sum(),'amount_Sales':data_merge_hrg.amount_Sales.sum()}

Выбросы

In [147]:
big_data = data_OCP_export[(data_OCP_export.sourceTable=='unifiedDealsBrazil_externalSalesOrders')&(data_OCP_export.salesUnitsId=='BRFTO')]
# big_data

In [149]:
if print_to_excel == True:
    Ouput_file = f'{str(date.today())}_deals_BR_vs_unOCP.xlsx'
    with pd.ExcelWriter(Ouput_file, engine='openpyxl') as writer:
        data_group.to_excel(writer, index=False, sheet_name='data_SQL')
        data_merge.to_excel(writer, index=False, sheet_name='BRFTO')
        data_merge_hrg.to_excel(writer, index=False, sheet_name='BRHRG')
        big_data.to_excel(writer, index=False, sheet_name='unifiedDealsBrazil_externalSalesOrders')

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [29]:
# query = f"""select distinct "sourceTable"
# from "RISKACCESS"."unifiedOcp" 
# """
# data_sourceTable = export_from_RISKCUSTOM(query)
# data_sourceTable

,sourceTable
0,flowDealsReportChanges
1,unifiedDealsMarketSuek
2,unifiedDealsBrazil_sapSalesOrders
3,unifiedDealsMarketSuek_inventory
4,unifiedDealsBrazil_inventory_sapSalesOrders
5,unifiedDealsBrazil_inventory_externalPurchases...
6,unifiedDealsBrazil_purchasesRecapBRHRG
7,unifiedDealsBrazil_inventory_externalSalesOrders
8,flowDealsReportChanges_inventory
9,bankAccountsBalance


In [ ]:
# unifiedDealsBrazil_purchasesRecapBRHRG
# unifiedDealsBrazil_externalPurchasesRecap
# unifiedDealsBrazil_externalSalesOrders
# unifiedDealsBrazil_sapSalesOrders
# flowDealsReportChanges